In [1]:
%load_ext autoreload
%autoreload 2
%aimport anndata
%aimport os
%aimport torch
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from typing import Dict, Union, List, Tuple
from torch.utils.data import Dataset
# Control UMAP numba warnings
import warnings; warnings.simplefilter('ignore')
import copy
import scvi

from scvi.models.scanvi import SCANVI
from scvi.inference import UnsupervisedTrainer, load_posterior, SemiSupervisedTrainer
from scvi import set_seed
from scvi.dataset import setup_anndata
# from scvi.dataset.dataset10X import Dataset10X

# Sets torch and numpy random seeds, run after all scvi imports
set_seed(0)


from scvi.dataset import cortex
from scvi.dataset.seqfish import seqfish, SeqfishDataset


/Users/galen/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [2]:
s = seqfish()

[2020-07-01 12:17:28,731] INFO - scvi.dataset._utils | File /Users/galen/scVI/galen/data/SeqFISH.xlsx already downloaded
[2020-07-01 12:17:28,732] INFO - scvi.dataset.seqfish | Loading seqfish dataset from /Users/galen/scVI/galen/data/SeqFISH.xlsx
> /Users/galen/scVI/scvi/dataset/seqfish.py(30)_load_seqfish_data()
-> gene_names = counts.values[:, 0].astype(str)


(Pdb)  n


> /Users/galen/scVI/scvi/dataset/seqfish.py(31)_load_seqfish_data()
-> adata = anndata.AnnData(pd.DataFrame(data=X, columns=gene_names))


(Pdb)  gene_names


array(["'Dmbx1'", "'Emx2'", "'Uncx'", "'Paxip1'", "'Ctnnb1'", "'Prdm1'",
       "'Rybp'", "'Nfkb2'", "'Tfdp2'", "'Grhl1'", "'Sp8'", "'Irf2'",
       "'Zfp287'", "'Esr2'", "'Zfp128'", "'Vav1'", "'Sp1'", "'Ppargc1b'",
       "'Sp7'", "'Pin1'", "'Nfya'", "'Vsx1'", "'Klf1'", "'Vsx2'",
       "'Mybl1'", "'Mybl2'", "'Rnf2'", "'Blzf1'", "'Topors'", "'Nr3c2'",
       "'Nfia'", "'Taf6l'", "'Nr4a3'", "'Hoxd12'", "'Hoxd13'", "'Ttf1'",
       "'Sox9'", "'Nr2e1'", "'Polr2b'", "'Hltf'", "'Sox6'", "'Pbx3'",
       "'Sox5'", "'Foxa1'", "'Cdc5l'", "'Cebpg'", "'Ciita'", "'Rest'",
       "'Ets1'", "'Mafk'", "'Tbx15'", "'Scml2'", "'Myb'", "'Clock'",
       "'Rbpj'", "'Foxc1'", "'Zfp422'", "'Pias3'", "'Runx1'", "'Ppara'",
       "'Relb'", "'Vdr'", "'Cdc6'", "'Arid3a'", "'Lhx1'", "'Hoxb8'",
       "'Hoxb9'", "'Hic1'", "'Lhx6'", "'Six4'", "'Hoxb3'", "'Zfp263'",
       "'Cbfa2t3'", "'Ehf'", "'Nhlh1'", "'Gata6'", "'Gata4'", "'Gata5'",
       "'Lpp'", "'Nfe2l3'", "'Nfe2l2'", "'Tmf1'", "'Gli1'", "'Tbx2'",
      

(Pdb)  c


Transforming to str index.


[2020-07-01 12:17:58,847] INFO - scvi.dataset.seqfish | Finished loading seqfish dataset
[2020-07-01 12:17:58,871] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-07-01 12:17:58,872] INFO - scvi.dataset._anndata | No batch_key inputted, assuming all cells are same batch
[2020-07-01 12:17:58,876] INFO - scvi.dataset._anndata | No label_key inputted, assuming all cells have same label
[2020-07-01 12:17:58,879] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-07-01 12:17:58,885] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 3585 cells, 248 genes, and 1 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']


In [3]:
s

AnnData object with n_obs × n_vars = 3585 × 248 
    obs: '_scvi_batch', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    uns: 'scvi_data_registry', 'scvi_summary_stats'

GeneExpressionDataset object with n_cells x nb_genes = 248 x 3585
    cell_attribute_names: 'batch_indices', 'labels', 'local_means', 'local_vars'
    cell_categorical_attribute_names: 'batch_indices', 'labels'

In [38]:
c

AnnData object with n_obs × n_vars = 3005 × 19972 
    obs: 'labels', 'precise_labels', 'cell_type', '_scvi_batch', '_scvi_local_l_mean', '_scvi_local_l_var'
    uns: 'scvi_data_registry', 'scvi_summary_stats'

... storing 'precise_labels' as categorical
... storing 'cell_type' as categorical


In [11]:
d = Dataset10X("pbmc8k")

http://cf.10xgenomics.com/samples/cell-exp/2.1.0/pbmc8k/pbmc8k_raw_gene_bc_matrices_h5.h5
[2020-07-01 10:46:20,723] INFO - scvi.dataset.dataset | File data/10X/pbmc8k/raw_gene_bc_matrices_h5.h5 already downloaded
> /Users/galen/scVI/scvi/dataset/dataset10X.py(164)__init__()
-> # setup_anndata(adata, )


(Pdb)  exit


BdbQuit: 

In [7]:
d.adata

AnnData object with n_obs × n_vars = 8381 × 33694 
    var: 'gene_ids'

In [4]:
from scvi.dataset.pbmc import PurifiedPBMCDataset


In [6]:
p = PurifiedPBMCDataset()

[2020-06-30 00:25:37,794] INFO - scvi.dataset.dataset | File /Users/galen/scVI/galen/data/cd4_t_helper/filtered_gene_bc_matrices.tar.gz already downloaded
[2020-06-30 00:25:37,795] INFO - scvi.dataset.dataset10X | Preprocessing dataset
[2020-06-30 00:25:37,799] INFO - scvi.dataset.dataset10X | Extracting tar file
[2020-06-30 00:25:46,513] INFO - scvi.dataset.dataset10X | Finished preprocessing dataset
[2020-06-30 00:25:46,634] WARNING - scvi.dataset.dataset | Gene names are not unique.
[2020-06-30 00:25:46,635] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-30 00:25:46,637] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-30 00:25:46,807] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-30 00:25:46,877] INFO - scvi.dataset.dataset | Downsampled from 11213 to 11213 cells
[2020-06-30 00:25:46,898] INFO - scvi.dataset.dataset | File /Users/galen/scVI/galen/data/regulatory_t/filtered_gene_bc_matrices.tar.gz already 

In [11]:
from scvi.dataset.cortex import CortexDataset
from scvi.dataset.hemato import HematoDataset
from scvi.dataset.seqfish import SeqfishDataset

In [12]:
cd = SeqfishDataset()

actual
https://www.cell.com/cms/attachment/2080562255/2072099886/mmc6.xlsx
[2020-06-30 12:56:53,099] INFO - scvi.dataset.dataset | Downloading file at /Users/galen/scVI/galen/data/SeqFISH.xlsx
[2020-06-30 12:56:57,778] INFO - scvi.dataset.seqfish | Preprocessing dataset
[2020-06-30 12:57:06,549] INFO - scvi.dataset.seqfish | Finished preprocessing dataset
[2020-06-30 12:57:06,577] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-30 12:57:06,578] INFO - scvi.dataset.dataset | Remapping labels to [0,N]


In [13]:
cd = cd.to_anndata()
cd.write(filename = 'seqfish.h5ad')

Transforming to str index.
Transforming to str index.
... storing 'cell_types' as categorical


In [15]:
from scvi.dataset import setup_anndata
setup_anndata(cd, batch_key = 'batch_indices', labels_key = "precise_labels")

[2020-06-30 12:10:12,309] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-06-30 12:10:12,310] INFO - scvi.dataset._anndata | Using batches from adata.obs["batch_indices"]
[2020-06-30 12:10:12,312] INFO - scvi.dataset._anndata | Using labels from adata.obs["precise_labels"]
[2020-06-30 12:10:12,317] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-06-30 12:10:12,329] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 3005 cells, 558 genes, and 1 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']


In [18]:
import anndata
def cortex():
    adata =  anndata.read('cortex.h5ad')
    return setup_anndata(adata, batch_key = 'batch_indices', labels_key = "precise_labels")



In [1]:
from scvi.dataset.dataset10X import Dataset10X

/Users/galen/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [ ]:
    "2.1.0": ["pbmc8k", "pbmc4k", "t_3k", "t_4k", "neuron_9k"],
    "3.0.0": [
        "pbmc_1k_protein_v3",
        "pbmc_10k_protein_v3",
        "malt_10k_protein_v3",
        "pbmc_1k_v2",
        "pbmc_1k_v3",
        "pbmc_10k_v3",
        "hgmm_1k_v2",
        "hgmm_1k_v3",
        "hgmm_5k_v3",
        "hgmm_10k_v3",
        "neuron_1k_v2",
        "neuron_1k_v3",
        "neuron_10k_v3",
        "heart_1k_v2",
        "heart_1k_v3",
        "heart_10k_v3",
    ],
    "3.1.0": ["5k_pbmc_protein_v3", "5k_pbmc_protein_v3_nextgem"],

http://cf.10xgenomics.com/samples/cell-exp/2.1.0/pbmc4k/pbmc4k_filtered_gene_bc_matrices.tar.gz
actual
http://cf.10xgenomics.com/samples/cell-exp/2.1.0/pbmc4k/pbmc4k_filtered_gene_bc_matrices.tar.gz
[2020-06-30 12:39:35,977] INFO - scvi.dataset.dataset | Downloading file at /Users/galen/scVI/galen/data/10X/pbmc4k/filtered_gene_bc_matrices.tar.gz
[2020-06-30 12:39:37,098] INFO - scvi.dataset.dataset10X | Preprocessing dataset
[2020-06-30 12:39:37,101] INFO - scvi.dataset.dataset10X | Extracting tar file
[2020-06-30 12:39:45,163] INFO - scvi.dataset.dataset10X | Finished preprocessing dataset
[2020-06-30 12:39:45,269] WARNING - scvi.dataset.dataset | Gene names are not unique.
[2020-06-30 12:39:45,270] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-30 12:39:45,271] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-30 12:39:45,398] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-30 12:39:45,473] INFO - scvi.dataset

In [5]:
d

GeneExpressionDataset object with n_cells x nb_genes = 1222 x 33538
    gene_attribute_names: 'gene_names'
    cell_attribute_names: 'labels', 'barcodes', 'local_means', 'local_vars', 'batch_indices'
    cell_categorical_attribute_names: 'labels', 'batch_indices'

In [8]:
d = Dataset10X("pbmc_1k_v3")

http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_v3/pbmc_1k_v3_filtered_feature_bc_matrix.tar.gz
[2020-06-30 12:47:41,753] INFO - scvi.dataset.dataset | File /Users/galen/scVI/galen/data/10X/pbmc_1k_v3/filtered_feature_bc_matrix.tar.gz already downloaded
[2020-06-30 12:47:41,754] INFO - scvi.dataset.dataset10X | Preprocessing dataset
[2020-06-30 12:47:46,700] INFO - scvi.dataset.dataset10X | Finished preprocessing dataset
[2020-06-30 12:47:46,757] WARNING - scvi.dataset.dataset | Gene names are not unique.
[2020-06-30 12:47:46,758] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-30 12:47:46,759] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-30 12:47:46,822] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-30 12:47:46,849] INFO - scvi.dataset.dataset | Downsampled from 1222 to 1222 cells


In [9]:
d

GeneExpressionDataset object with n_cells x nb_genes = 1222 x 33538
    gene_attribute_names: 'gene_names'
    cell_attribute_names: 'labels', 'barcodes', 'local_means', 'local_vars', 'batch_indices'
    cell_categorical_attribute_names: 'labels', 'batch_indices'

In [34]:
available_datasets = {
    # these can made manually
    "1.1.0": [
        "frozen_pbmc_donor_a",
        "frozen_pbmc_donor_b",
        "frozen_pbmc_donor_c",
        "fresh_68k_pbmc_donor_a",
        "cd14_monocytes",
        "b_cells",
        "cd34",
        "cd56_nk",
        "cd4_t_helper",
        "regulatory_t",
        "naive_t",
        "memory_t",
        "cytotoxic_t",
        "naive_cytotoxic",
    ],
    # all of these can be h5 files
    "2.1.0": ["pbmc8k", "pbmc4k", "t_3k", "t_4k", "neuron_9k"],
    "3.0.0": [
        "pbmc_1k_protein_v3",
        "pbmc_10k_protein_v3",
        "malt_10k_protein_v3",
        "pbmc_1k_v2",
        "pbmc_1k_v3",
        "pbmc_10k_v3",
        "hgmm_1k_v2",
        "hgmm_1k_v3",
        "hgmm_5k_v3",
        "hgmm_10k_v3",
        "neuron_1k_v2",
        "neuron_1k_v3",
        "neuron_10k_v3",
        "heart_1k_v2",
        "heart_1k_v3",
        "heart_10k_v3",
    ],
    "3.1.0": ["5k_pbmc_protein_v3", "5k_pbmc_protein_v3_nextgem"],
}

dataset_to_group = dict(
    [
        (dataset_name, group)
        for group, list_datasets in available_datasets.items()
        for dataset_name in list_datasets
    ]
)

group_to_url_skeleton = {
    "1.1.0": "http://cf.10xgenomics.com/samples/cell-exp/{}/{}/{}_{}_gene_bc_matrices.tar.gz",
    "2.1.0": "http://cf.10xgenomics.com/samples/cell-exp/{}/{}/{}_{}_gene_bc_matrices.tar.gz",
    "3.0.0": "http://cf.10xgenomics.com/samples/cell-exp/{}/{}/{}_{}_feature_bc_matrix.tar.gz",
    "3.1.0": "http://cf.10xgenomics.com/samples/cell-exp/{}/{}/{}_{}_feature_bc_matrix.tar.gz",
}

group_to_filename_skeleton = {
    "1.1.0": "{}_gene_bc_matrices.tar.gz",
    "2.1.0": "{}_gene_bc_matrices.tar.gz",
    "3.0.0": "{}_feature_bc_matrix.tar.gz",
    "3.1.0": "{}_feature_bc_matrix.tar.gz",
}

available_specification = ["filtered", "raw"]

dataset_name = "pbmc8k"
group = dataset_to_group[dataset_name]
url_skeleton = group_to_url_skeleton[group]
url = url_skeleton.format(group, dataset_name, dataset_name, 'filtered')

In [35]:
url

'http://cf.10xgenomics.com/samples/cell-exp/2.1.0/pbmc8k/pbmc8k_filtered_gene_bc_matrices.tar.gz'

In [3]:
import scanpy
scanpy.read_10x_h5("http://cf.10xgenomics.com/samples/cell-exp/2.1.0/pbmc8k/pbmc8k_filtered_gene_bc_matrices.tar.gz")


OSError: ``http://cf.10xgenomics.com/samples/cell-exp/2.1.0/pbmc8k/pbmc8k_filtered_gene_bc_matrices.tar.gz`` does not exist

In [19]:
c = cortex()

[2020-06-30 12:11:12,196] INFO - scvi.dataset._anndata | Using data from adata.X
[2020-06-30 12:11:12,197] INFO - scvi.dataset._anndata | Using batches from adata.obs["batch_indices"]
[2020-06-30 12:11:12,198] INFO - scvi.dataset._anndata | Using labels from adata.obs["precise_labels"]
[2020-06-30 12:11:12,200] INFO - scvi.dataset._anndata | Computing library size prior per batch
[2020-06-30 12:11:12,210] INFO - scvi.dataset._anndata | Successfully registered anndata object containing 3005 cells, 558 genes, and 1 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']


In [10]:
c

AnnData object with n_obs × n_vars = 3005 × 558 
    obs: 'cell_types', 'precise_labels', 'batch_indices'

In [13]:
cd.uns['cell_measurements_col_mappings']


{}